In [ ]:
# Importing Libraries
from sklearnex import patch_sklearn #Improves sklearn alghoritms performance
patch_sklearn()
import sklearn
print('scikit-learn version\n', sklearn.__version__)
from sklearn.model_selection import ParameterGrid, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OutputCodeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import ComplementNB
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#Set seed
seed=41

In [ ]:
classif_level = 1

# Load Data

In [ ]:
feature_names = pickle.load(open('Dataset/feature_names.pkl', 'rb'))
X_train = pickle.load(open(f'Dataset/X_train.pkl', 'rb'))
X_train_scal = pickle.load(open(f'Dataset/X_train_scal.pkl', 'rb'))
y_train = pickle.load(open(f'Dataset/y_train.pkl', 'rb'))
kings = pickle.load(open('Dataset/all_categories.pkl', 'rb'))['Kingdom']

In [ ]:
for i, king in enumerate(kings):
    print(i, king)

## Train and Tune classifiers

### Random Forests

In [ ]:
for i, king in enumerate(kings):
    print(king)
    for f_sel in ['all', 'sel']:
        if f_sel == 'sel':
            sel_f = pickle.load(open('Results/selected_features.pkl', 'rb'))['Superclass'][king]['selected features']
            f_index = [feature_names.index(feature) for feature in sel_f]
            X_train_ = X_train[y_train[:, classif_level-1]==king][:, f_index]
            grid = {'class_weight': [None, 'balanced', 'balanced_subsample'],
                'criterion' : ['gini', 'entropy'],
                'max_features' : range(1, len(sel_f)+1)}
        else:
            X_train_ = X_train[y_train[:, classif_level-1]==king]
            grid = {'class_weight': [None, 'balanced', 'balanced_subsample'],
                'criterion' : ['gini', 'entropy'],
                'max_features' : [1, 2, 3, 5, 7, 10, 15, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 133]}
        y_train_ = y_train[y_train[:, classif_level-1]==king][:, classif_level]
        
        estimator = RandomForestClassifier(random_state=seed, n_jobs=-1)
        gs = GridSearchCV(estimator, param_grid=grid, scoring=['f1_macro', 'f1_micro'], refit='f1_macro', cv=3, verbose=3, 
                          error_score='raise', return_train_score=True, n_jobs=-1)
        gs.fit(X_train_, y_train_)
        pickle.dump(gs, open(f'Models/Superclass/{i}_RF_{f_sel}_feat.pkl', 'wb'))

In [ ]:
for i, king in enumerate(kings):
    print(king)
    for f_sel in ['all', 'sel']:
        print(f_sel)
        gs = pickle.load(open(f'Models/Superclass/{i}_RF_{f_sel}_feat.pkl', 'rb'))
        print('f1_macro_val ->', gs.cv_results_['mean_test_f1_macro'][gs.best_index_])
        print('f1_macro_train ->', gs.cv_results_['mean_train_f1_macro'][gs.best_index_])
        print('f1_micro_val ->', gs.cv_results_['mean_test_f1_micro'][gs.best_index_])
        print('f1_micro_train ->', gs.cv_results_['mean_train_f1_micro'][gs.best_index_])

Take 6 best estimators, and make pruning

In [ ]:
gs = pickle.load(open(f'Models/Superclass/1_RF_all_feat.pkl', 'rb'))
a = gs.cv_results_['mean_test_f1_macro']

In [ ]:
gs = pickle.load(open(f'Models/Superclass/1_RF_sel_feat.pkl', 'rb'))
b = gs.cv_results_['mean_test_f1_macro']

In [ ]:
c = np.concatenate((a, b), axis=0)

In [ ]:
ind = np.argpartition(c, -6)[-6:]
ind

In [ ]:
c[ind]

first gs has 120 comb, therefore none of its estimator are the best (120 index is the first one of the second gs)

In [ ]:
ind-120

by order: [200, 0, 51, 50, 53, 56]

In [ ]:
gs = pickle.load(open(f'Models/Superclass/1_RF_sel_feat.pkl', 'rb'))
best_params = [gs.cv_results_['params'][index] for index in [200, 0, 51, 50, 53, 56]]

In [ ]:
best_params

In [ ]:
grid = {'ccp_alpha': [0, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1]}
sel_f = pickle.load(open('Results/selected_features.pkl', 'rb'))['Superclass'][king]['selected features']
f_index = [feature_names.index(feature) for feature in sel_f]
X_train_ = X_train[y_train[:, classif_level-1]==king][:, f_index]
y_train_ = y_train[y_train[:, classif_level-1]==king][:, classif_level]

In [ ]:
for i, params in enumerate(best_params):
    estimator = RandomForestClassifier(random_state=seed, n_jobs=-1)
    estimator.set_params(**params)
    gs = GridSearchCV(estimator, param_grid=grid, scoring=['f1_macro', 'f1_micro'], refit='f1_macro', cv=3, verbose=3, 
                          error_score='raise', return_train_score=True, n_jobs=-20)
    gs.fit(X_train_, y_train_)
    pickle.dump(gs, open(f'Models/Superclass/1_RF_post_pruning_{i}.pkl', 'wb'))

In [ ]:
results = []
for i in range(6):
    gs = pickle.load(open(f'Models/Superclass/1_RF_post_pruning_{i}.pkl', 'rb'))
    macro_test = gs.cv_results_['mean_test_f1_macro']
    macro_train = gs.cv_results_['mean_train_f1_macro']
    micro_test = gs.cv_results_['mean_test_f1_micro']
    micro_train = gs.cv_results_['mean_train_f1_micro']
    for z, x in enumerate([0, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1]):
        results.append({'Estimator': str(i+1), 'ccp_alpha': x, 'Subset': 'train f1-score macro',  'score': macro_train[z]})
        results.append({'Estimator': str(i+1), 'ccp_alpha': x, 'Subset': 'val f1-score macro',  'score': macro_test[z]})
        results.append({'Estimator': str(i+1), 'ccp_alpha': x, 'Subset': 'train f1-score micro',  'score': micro_train[z]})
        results.append({'Estimator': str(i+1), 'ccp_alpha': x, 'Subset': 'val f1-score micro',  'score': micro_test[z]})
results = pd.DataFrame(results)

In [ ]:
sns.set_theme(style='whitegrid', font_scale=1.1)

ax = sns.relplot(data=results, x='ccp_alpha', y='score', hue='Subset', col='Estimator', kind='line', 
                facet_kws={'sharey':False}, col_wrap=3, aspect=1.15)

ax.set(xscale='log')
sns.move_legend(ax, "lower center", bbox_to_anchor=(0.425, -0.05), title=None, ncol=4, fontsize=17)

plt.savefig('Plots/Pruning_effect', bbox_inches='tight', transparent=True, dpi=500)
plt.show()

In [ ]:
sns.set_theme(style='whitegrid', font_scale=1.1)

ax = sns.relplot(data=results, x='ccp_alpha', y='score', hue='Subset', col='Estimator', kind='line', 
                facet_kws={'sharey':False}, col_wrap=3, aspect=1.15)

ax.set(xscale='log', xlim=(1e-8, 1e-3), ylim=(0.4, 1))
sns.move_legend(ax, "lower center", bbox_to_anchor=(0.425, -0.05), title=None, ncol=4, fontsize=17)

plt.savefig('Plots/Pruning_effect_increased', bbox_inches='tight', transparent=True, dpi=500)
plt.show()

In [ ]:
results[(results['Estimator']=='1') & (results['Subset']=='val f1-score macro') ]

In [ ]:
results[(results['Estimator']=='2') & (results['Subset']=='val f1-score micro') ]

### Multiclass strategy (output code) - Organic compounds only

In [ ]:
king = 'Organic compounds'
i = 1
for f_sel in ['all', 'sel']:
    print(f_sel)
    if f_sel == 'sel':
        sel_f = pickle.load(open('Results/selected_features.pkl', 'rb'))['Superclass'][king]['selected features']
        f_index = [feature_names.index(feature) for feature in sel_f]
        X_train_ = X_train[y_train[:, classif_level-1]==king][:, f_index]
        grid = {'estimator__class_weight': [None, 'balanced', 'balanced_subsample'],
            'estimator__criterion' : ['gini', 'entropy'],
            'estimator__max_features' : range(1, len(sel_f)+1)
            'code_size': [0.25, 0.5, 0.75, 1, 1.25, 1.5, 1.75, 2, 2.5, 3, 3.5, 4, 4.5, 5]}
    else:
        X_train_ = X_train[y_train[:, classif_level-1]==king]
        grid = {'estimator__class_weight': [None, 'balanced', 'balanced_subsample'],
            'estimator__criterion' : ['gini', 'entropy'],
            'estimator__max_features' : [1, 2, 3, 5, 7, 10, 15, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 133],
            'code_size': [0.25, 0.5, 0.75, 1, 1.25, 1.5, 1.75, 2, 2.5, 3, 3.5, 4, 4.5, 5]}
    y_train_ = y_train[y_train[:, classif_level-1]==king][:, classif_level]
    estimator = OutputCodeClassifier(RandomForestClassifier(random_state=seed, n_jobs=-1),
                                     random_state=seed, n_jobs=-1)
    gs = GridSearchCV(estimator, param_grid=grid, scoring=['f1_macro', 'f1_micro'], refit='f1_macro', cv=3, verbose=3, 
                      error_score='raise', return_train_score=True, n_jobs=-1)
    gs.fit(X_train_, y_train_)
    pickle.dump(gs, open(f'Models/Superclass/1_RF_OutputCode_{f_sel}_feat.pkl', 'wb'))

In [ ]:
for f_sel in ['all', 'sel']:
    print(f_sel)
    gs = pickle.load(open(f'Models/Superclass/1_RF_OutputCode_{f_sel}_feat.pkl', 'rb'))
    print(gs.best_estimator_)
    print('f1_macro_val ->', gs.cv_results_['mean_test_f1_macro'][gs.best_index_])
    print('f1_macro_train ->', gs.cv_results_['mean_train_f1_macro'][gs.best_index_])
    print('f1_micro_val ->', gs.cv_results_['mean_test_f1_micro'][gs.best_index_])
    print('f1_micro_train ->', gs.cv_results_['mean_train_f1_micro'][gs.best_index_])

### KNN

In [ ]:
grid = {'n_neighbors': range(1, 31), 'weights' : ['uniform', 'distance'],
        'metric':['manhattan', 'euclidean', 'cosine']}

In [ ]:
for i, king in enumerate(kings):
    print(king)
    for f_sel in ['all', 'sel']:
        print(f_sel)
        if f_sel == 'sel':
            sel_f = pickle.load(open('Results/selected_features.pkl', 'rb'))['Superclass'][king]['selected features']
            f_index = [feature_names.index(feature) for feature in sel_f]
            X_train_ = X_train_scal[y_train[:, classif_level-1]==king][:, f_index]
        else:
            X_train_ = X_train_scal[y_train[:, classif_level-1]==king]
            
        y_train_ = y_train[y_train[:, classif_level-1]==king][:, classif_level]
        
        estimator = KNeighborsClassifier(algorithm = 'brute', n_jobs=-1)
        gs = GridSearchCV(estimator, param_grid=grid, scoring=['f1_macro', 'f1_micro'], refit='f1_macro', cv=3, verbose=3, 
                          error_score='raise', return_train_score=True,  n_jobs=-1, pre_dispatch=10)
        gs.fit(X_train_, y_train_)
        pickle.dump(gs, open(f'Models/Superclass/{i}_KNN_{f_sel}_feat.pkl', 'wb'))

In [ ]:
for i, king in enumerate(kings):
    print(king)
    for f_sel in ['all', 'sel']:
        print(f_sel)
        gs = pickle.load(open(f'Models/Superclass/{i}_KNN_{f_sel}_feat.pkl', 'rb'))
        print(f_sel)
        print('f1_macro_test ->', gs.cv_results_['mean_test_f1_macro'][gs.best_index_])
        print('f1_macro_train ->', gs.cv_results_['mean_train_f1_macro'][gs.best_index_])
        print('f1_micro_test ->', gs.cv_results_['mean_test_f1_micro'][gs.best_index_])
        print('f1_micro_train ->', gs.cv_results_['mean_train_f1_micro'][gs.best_index_])
        print(gs)

### Logistic Regression

In [ ]:
grid = [{'penalty': ['l1'],
         'C': [0.01, 0.1, 1, 10, 100],
         'fit_intercept': [True, False],
         'class_weight': [None, 'balanced'],
         'multi_class':['ovr', 'multinomial'],
         'solver': ['saga']},
        {'penalty': ['elasticnet'],
         'C': [0.01, 0.1, 1, 10, 100],
         'fit_intercept': [True, False],
         'class_weight': [None, 'balanced'],
         'multi_class':['ovr', 'multinomial'],
         'l1_ratio': [0.25, 0.5, 0.75],
         'solver': ['saga']},
        {'penalty': ['l2'],
         'C': [0.01, 0.1, 1, 10, 100],
         'fit_intercept': [True, False],
         'class_weight': [None, 'balanced'],
         'multi_class':['ovr', 'multinomial'],
         'solver': ['sag', 'saga']},
       {'penalty': ['none'],
         'fit_intercept': [True, False],
         'class_weight': [None, 'balanced'],
         'multi_class':['ovr', 'multinomial'],
         'solver': ['sag', 'saga']}]

In [ ]:
for i, king in enumerate(kings):
    print(king)
    if i == 0:
        max_iter=5000
    else:
        max_iter=100
    for f_sel in ['all', 'sel']:
        print(f_sel)
        if f_sel == 'sel':
            sel_f = pickle.load(open('Results/selected_features.pkl', 'rb'))['Superclass'][king]['selected features']
            f_index = [feature_names.index(feature) for feature in sel_f]
            X_train_ = X_train_scal[y_train[:, classif_level-1]==king][:, f_index]
        else:
            X_train_ = X_train_scal[y_train[:, classif_level-1]==king]
            
        y_train_ = y_train[y_train[:, classif_level-1]==king][:, classif_level]
        
        estimator = LogisticRegression(max_iter=max_iter, tol=1e-6, random_state=seed, n_jobs=-1)
        gs = GridSearchCV(estimator, param_grid=grid, scoring=['f1_macro', 'f1_micro'], refit='f1_macro', cv=3, verbose=3, 
                          error_score='raise', return_train_score=True, n_jobs=-1)
        gs.fit(X_train_, y_train_)
        pickle.dump(gs, open(f'Models/Superclass/{i}_LR_{f_sel}_feat.pkl', 'wb'))

In [ ]:
for i, king in enumerate(kings):
    print(king)
    for f_sel in ['all', 'sel']:
        print(f_sel)
        gs = pickle.load(open(f'Models/Superclass/{i}_LR_{f_sel}_feat.pkl', 'rb'))
        print('f1_macro_test ->', gs.cv_results_['mean_test_f1_macro'][gs.best_index_])
        print('f1_macro_train ->', gs.cv_results_['mean_train_f1_macro'][gs.best_index_])
        print('f1_micro_test ->', gs.cv_results_['mean_test_f1_micro'][gs.best_index_])
        print('f1_micro_train ->', gs.cv_results_['mean_train_f1_micro'][gs.best_index_])

### SVM

In [ ]:
grid = [{'penalty': ['l1', 'l2'],
        'C': [0.01, 0.1, 1, 10, 100],
        'class_weight': [None, 'balanced']}]

In [ ]:
for i, king in enumerate(kings):
    print(king)
    for f_sel in ['all', 'sel']:
        print(f_sel)
        if f_sel == 'sel':
            sel_f = pickle.load(open('Results/selected_features.pkl', 'rb'))['Superclass'][king]['selected features']
            f_index = [feature_names.index(feature) for feature in sel_f]
            X_train_ = X_train_scal[y_train[:, classif_level-1]==king][:, f_index]
        else:
            X_train_ = X_train_scal[y_train[:, classif_level-1]==king]
            
        y_train_ = y_train[y_train[:, classif_level-1]==king][:, classif_level]
        
        estimator = LinearSVC(dual=False, loss='squared_hinge', multi_class='ovr', tol=1e-6, max_iter=5000, 
                              random_state=seed, verbose=3)
        gs = GridSearchCV(estimator, param_grid=grid, scoring=['f1_macro', 'f1_micro'], refit='f1_macro', cv=3, verbose=3, 
                          error_score='raise', return_train_score=True, n_jobs=-1)
        gs.fit(X_train_, y_train_)
        pickle.dump(gs, open(f'Models/Superclass/{i}_SVM_{f_sel}_feat.pkl', 'wb'))

In [ ]:
for i, king in enumerate(kings):
    print(king)
    for f_sel in ['all', 'sel']:
        print(f_sel)
        gs = pickle.load(open(f'Models/Superclass/{i}_SVM_{f_sel}_feat.pkl', 'rb'))
        print('f1_macro_test ->', gs.cv_results_['mean_test_f1_macro'][gs.best_index_])
        print('f1_macro_train ->', gs.cv_results_['mean_train_f1_macro'][gs.best_index_])
        print('f1_micro_test ->', gs.cv_results_['mean_test_f1_micro'][gs.best_index_])
        print('f1_micro_train ->', gs.cv_results_['mean_train_f1_micro'][gs.best_index_])

### ComplementNB

In [ ]:
grid = {'alpha': [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100], 
        'norm': [True, False]}

In [ ]:
for i, king in enumerate(kings):
    print(king)
    for f_sel in ['all', 'sel']:
        print(f_sel)
        if f_sel == 'sel':
            sel_f = pickle.load(open('Results/selected_features.pkl', 'rb'))['Superclass'][king]['selected features']
            f_index = [feature_names.index(feature) for feature in sel_f]
            X_train_ = X_train_scal[y_train[:, classif_level-1]==king][:, f_index]
        else:
            X_train_ = X_train_scal[y_train[:, classif_level-1]==king]
            
        y_train_ = y_train[y_train[:, classif_level-1]==king][:, classif_level]
        
        estimator = ComplementNB()
        gs = GridSearchCV(estimator, param_grid=grid, scoring=['f1_macro', 'f1_micro'], refit='f1_macro', cv=3, verbose=3, 
                          error_score='raise', return_train_score=True, n_jobs=-1)
        gs.fit(X_train_, y_train_)
        pickle.dump(gs, open(f'Models/Superclass/{i}_NB_{f_sel}_feat.pkl', 'wb'))

In [ ]:
for i, king in enumerate(kings):
    print(king)
    for f_sel in ['all', 'sel']:
        print(f_sel)
        gs = pickle.load(open(f'Models/Superclass/{i}_NB_{f_sel}_feat.pkl', 'rb'))
        print('f1_macro_test ->', gs.cv_results_['mean_test_f1_macro'][gs.best_index_])
        print('f1_macro_train ->', gs.cv_results_['mean_train_f1_macro'][gs.best_index_])
        print('f1_micro_test ->', gs.cv_results_['mean_test_f1_micro'][gs.best_index_])
        print('f1_micro_train ->', gs.cv_results_['mean_train_f1_micro'][gs.best_index_])

### Load GS scores

In [ ]:
scores_df = []
params_df = []
for i, king in enumerate(kings):
    #Select if result is better with or without f selection
    scores_rows = [{'Classifier':king, 'Feature selection': 'No', 'F1-score average': 'macro'}, 
                   {'Classifier':king, 'Feature selection': 'No', 'F1-score average': 'micro'},
                   {'Classifier':king, 'Feature selection': 'Yes', 'F1-score average': 'macro'}, 
                   {'Classifier':king, 'Feature selection': 'Yes', 'F1-score average': 'micro'}]
    params_row = {'Classifier':king}
    for alg in ['RF', 'KNN', 'LR', 'SVM', 'NB']:
        f_sel_gs = {}
        for f_sel in ['all', 'sel']:
            f_sel_gs[f_sel] = pickle.load(open(f'Models/Superclass/{i}_{alg}_{f_sel}_feat.pkl', 'rb'))
        score_all = f_sel_gs['all'].cv_results_[f'mean_test_f1_macro'][f_sel_gs['all'].best_index_]
        score_sel = f_sel_gs['sel'].cv_results_[f'mean_test_f1_macro'][f_sel_gs['sel'].best_index_]
        if score_all >= score_sel:
            f_sel = 'all'
            f = 'No'
        else:
            f_sel = 'sel'
            f = 'Yes'
        gs = f_sel_gs[f_sel]
        #Save scores
        for z, f_sel in enumerate(['all', 'sel']):
            z=z*2
            scores_rows[0 + z][f'{alg} (val)'] = f_sel_gs[f_sel].cv_results_[f'mean_test_f1_macro'][f_sel_gs[f_sel].best_index_]
            scores_rows[0 + z][f'{alg} (train)'] = f_sel_gs[f_sel].cv_results_[f'mean_train_f1_macro'][f_sel_gs[f_sel].best_index_]
            scores_rows[1 + z][f'{alg} (val)'] = f_sel_gs[f_sel].cv_results_[f'mean_test_f1_micro'][f_sel_gs[f_sel].best_index_]
            scores_rows[1 + z][f'{alg} (train)'] = f_sel_gs[f_sel].cv_results_[f'mean_train_f1_micro'][f_sel_gs[f_sel].best_index_]
        #Save parameters
        params = gs.cv_results_['params'][gs.best_index_]
        params_row[f'{alg} (Feature Selection)'] = f
        for param_name, param in params.items():
            params_row[f'{alg} ({param_name})'] = param
    scores_df.extend(scores_rows)
    params_df.append(params_row)    
scores_df = pd.DataFrame(scores_df).set_index(['Classifier', 'Feature selection', 'F1-score average'])
params_df = pd.DataFrame(params_df).set_index(['Classifier'])

In [ ]:
with pd.ExcelWriter('Results/GS_scores.xlsx', mode='a') as writer:  
    scores_df.to_excel(writer, sheet_name='Superclass')
scores_df

In [ ]:
with pd.ExcelWriter('Results/GS_best_comb_params.xlsx', mode='a') as writer:  
    params_df.to_excel(writer, sheet_name='Superclass')
params_df